In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import chardet


Imagina que eres un investigador en el mundo de Pokémon y has recibido una base de datos con información desactualizada y desordenada sobre avistamientos de Pokémon. Tu misión es limpiar y actualizar esta base de datos para que pueda ser utilizada en un estudio sobre la población de Pokémon en la región.

Datos Iniciales

Los datos iniciales contienen las siguientes columnas:

    SightingDate: Fecha del avistamiento.
    TrainerID: Identificación del entrenador que reportó el avistamiento.
    PokemonName: Nombre del Pokémon avistado.
    CP: Puntos de combate del Pokémon reportado.
    HP: Puntos de salud del Pokémon reportado.
    Type: Tipo del Pokémon.
    Weather: Clima durante el avistamiento.

# 1 Carga de Datos

In [5]:
def find_encoding(fname):
    r_file = open(fname, 'rb').read()
    result = chardet.detect(r_file)
    return result['encoding']

encoding = find_encoding('pokemon.csv')
print(encoding)

path_datos = 'pokemon.csv'
# TODO completa el codigo para cargar los datos
df = pd.read_csv(path_datos, encoding= encoding)

print(df)

ISO-8859-1
               SightingDate    SightingTimeUTC TrainerID PokémonName       CP  \
0  2023-11-08T14:00:00+0000  2023-11-08T14:00Z     TR123     Pikachu      500   
1  2023-07-12T09:30:00+0100  2023-07-12T08:30Z     TR456       Eevee  MISSING   
2  2023/02/23T22:15:00+0900  2023-02-23T13:15Z     TR789    Magicarp    1,000   
3  2023-04-30T06:45:00-0400  2023-04-30T10:45Z     TR101      Gengar      800   
4  2023-08-15T16:00:00+1000                NaN     TR102   Bulbasaur      750   

   HP          Type        Weather  
0  35      Electric          Clear  
1  55        Normal         Cloudy  
2  10         Water           Rain  
3  45         Ghost  PARTLY_CLOUDY  
4  50  Grass/Poison          Sunny  


# Limpieza de Datos

## 1 Normalizacion de Zonas Horarias

Normaliza la columna `'SightingTimeUTC'` a la zona horaria UTC y convierte `'SightingDate'` al mismo formato de tiempo.


In [6]:
df['SightingTimeUTC'] = pd.to_datetime(df['SightingTimeUTC'], format="%Y-%m-%dT%H:%MZ").dt.tz_localize('America/Mexico_City')
print(df['SightingTimeUTC'])
df['SightingDate'] = df['SightingDate'].str.replace('/', '-')
df['SightingDate'] = df['SightingDate'].str.replace(r'(\d{2}:\d{2}:\d{2})-', r'\1+', regex=True)
#Viene de la parte de hacer un respaldo de tiempo local
df['SightingTimeLocal'] = pd.to_datetime(df['SightingDate'])
print(df['SightingTimeLocal'])
df['SightingDate'] = pd.to_datetime(df['SightingDate'], utc=True).dt.tz_convert('America/Mexico_City')
print(df['SightingDate'])

0   2023-11-08 14:00:00-06:00
1   2023-07-12 08:30:00-06:00
2   2023-02-23 13:15:00-06:00
3   2023-04-30 10:45:00-06:00
4                         NaT
Name: SightingTimeUTC, dtype: datetime64[ns, America/Mexico_City]
0    2023-11-08 14:00:00+00:00
1    2023-07-12 09:30:00+01:00
2    2023-02-23 22:15:00+09:00
3    2023-04-30 06:45:00+04:00
4    2023-08-15 16:00:00+10:00
Name: SightingTimeLocal, dtype: object
0   2023-11-08 08:00:00-06:00
1   2023-07-12 02:30:00-06:00
2   2023-02-23 07:15:00-06:00
3   2023-04-29 20:45:00-06:00
4   2023-08-15 00:00:00-06:00
Name: SightingDate, dtype: datetime64[ns, America/Mexico_City]


Compara si la fecha de la columna `'SightingDate'` coincide con la fecha en `'SightingTimeUTC'` una vez normalizada.


In [7]:
# Pista: Puedes crear una columna nueva para el resultado de la comparación.
df['DatesMatch'] = df['SightingDate'].dt.date == df['SightingTimeUTC'].dt.date
print (df)

               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 08:00:00-06:00 2023-11-08 14:00:00-06:00     TR123     Pikachu   
1 2023-07-12 02:30:00-06:00 2023-07-12 08:30:00-06:00     TR456       Eevee   
2 2023-02-23 07:15:00-06:00 2023-02-23 13:15:00-06:00     TR789    Magicarp   
3 2023-04-29 20:45:00-06:00 2023-04-30 10:45:00-06:00     TR101      Gengar   
4 2023-08-15 00:00:00-06:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather          SightingTimeLocal  \
0      500  35      Electric          Clear  2023-11-08 14:00:00+00:00   
1  MISSING  55        Normal         Cloudy  2023-07-12 09:30:00+01:00   
2    1,000  10         Water           Rain  2023-02-23 22:15:00+09:00   
3      800  45         Ghost  PARTLY_CLOUDY  2023-04-30 06:45:00+04:00   
4      750  50  Grass/Poison          Sunny  2023-08-15 16:00:00+10:00   

   DatesMatch  
0        True  
1        True  
2        True  
3       False  


Ajusta `'SightingTimeUTC'` a la zona horaria local de cada entrenador y crea una columna `'SightingTimeLocal'`.

In [11]:
# Se creo la columna antes de la comparación para poder ver los resultados
print(df['SightingTimeLocal'])

0    2023-11-08 14:00:00+00:00
1    2023-07-12 09:30:00+01:00
2    2023-02-23 22:15:00+09:00
3    2023-04-30 06:45:00+04:00
4    2023-08-15 16:00:00+10:00
Name: SightingTimeLocal, dtype: object


Calcula el tiempo transcurrido desde el momento del avistamiento hasta `'ahora'` (tu hora local) y crea una columna `'TimeSinceSighting'`.

In [12]:
from datetime import datetime

now = pd.Timestamp(datetime.now(), tz='UTC').tz_convert('America/Mexico_City')
df['TimeSinceSighting'] = now - df['SightingDate']
print(df['TimeSinceSighting'])

0     1 days 03:43:20.054486
1   120 days 09:13:20.054486
2   259 days 04:28:20.054486
3   193 days 14:58:20.054486
4    86 days 11:43:20.054486
Name: TimeSinceSighting, dtype: timedelta64[ns]


## 2. Limpeiza de IDs

Llena los valores faltantes en `'TrainerID'` con el ID `'UNKNOWN'`.


In [13]:
df['TrainerID'] = df['TrainerID'].fillna('UNKNOWN')
# Aunque en este caso no es necesario, pues no hay datos nulos en TrainerID
print(df['TrainerID'])

0    TR123
1    TR456
2    TR789
3    TR101
4    TR102
Name: TrainerID, dtype: object


## 3. Corrección de Nombres de Pokémon

Asegúrate de que los nombres de Pokémon estén capitalizados correctamente.


In [14]:
df['PokémonName'] = df['PokémonName'].str.capitalize()
# Solo el primer caracter de la cadena tiene mayuscula
print(df['PokémonName'])

0      Pikachu
1        Eevee
2     Magicarp
3       Gengar
4    Bulbasaur
Name: PokémonName, dtype: object


## 4. Conversión de 'CP' y 'HP' a Numéricos

Convierte `'CP'` y `'HP'` a valores numéricos, manejando los `'MISSING'` y comas como separadores de miles.

In [16]:
df['CP'] = df['CP'].astype(str)
df['HP'] = df['HP'].astype(str)
# Los volvemos cadenas para poder usar el replace
df['CP'] = pd.to_numeric(df['CP'].str.replace(',', ''), errors='coerce')
df['HP'] = pd.to_numeric(df['HP'].str.replace(',', ''), errors='coerce')
# Los volvemos a numericos para poder usarlos
print(df['CP'])
print(df['HP'])

0     500.0
1       NaN
2    1000.0
3     800.0
4     750.0
Name: CP, dtype: float64
0    35
1    55
2    10
3    45
4    50
Name: HP, dtype: int64


## 5. Estandarización de 'Type'

Divide la columna `'Type'` en `'PrimaryType'` y `'SecondaryType'` cuando hay dos tipos.


In [17]:
df[['PrimaryType', 'SecondaryType']] = df['Type'].str.split('/', expand=True)
print(df['PrimaryType'])
print(df['SecondaryType'])

0    Electric
1      Normal
2       Water
3       Ghost
4       Grass
Name: PrimaryType, dtype: object
0      None
1      None
2      None
3      None
4    Poison
Name: SecondaryType, dtype: object


## 6. Corrección del Clima

Estándariza la columna `'Weather'` para que todos los valores sean mayúsculas.


In [18]:
df['Weather'] = df['Weather'].str.upper()
print(df['Weather'])

0            CLEAR
1           CLOUDY
2             RAIN
3    PARTLY_CLOUDY
4            SUNNY
Name: Weather, dtype: object


# 3 Analisis de Datos

## Agrupaciones

Agrupa el DataFrame por `'Type'` y calcula la suma de `'CP'` para cada grupo.

In [19]:
groupByTiposCP = df.groupby('Type')['CP'].sum()
print(groupByTiposCP)

Type
Electric         500.0
Ghost            800.0
Grass/Poison     750.0
Normal             0.0
Water           1000.0
Name: CP, dtype: float64


Después de la suma, agrega una columna que calcule la media de `'HP'` por cada `'Type'`, pero solo para aquellos Pokémon cuyo `'CP'` sea mayor que el promedio de `'CP'` de todo el DataFrame.

In [21]:
promedio_CP = df['CP'].mean()

dfAltoCP = df[df['CP'] > promedio_CP]

groupByTiposHP = dfAltoCP.groupby('Type')['HP'].mean()

result = pd.DataFrame({'CP_Sum': groupByTiposCP, 'HP_Mean': groupByTiposHP})
print(result)

              CP_Sum  HP_Mean
Type                         
Electric       500.0      NaN
Ghost          800.0     45.0
Grass/Poison   750.0      NaN
Normal           0.0      NaN
Water         1000.0     10.0
